In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
ticker = "TSLA"

In [3]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import Window
import pandas as pd
import numpy as np
import pyarrow
import yfinance as yf
from pyspark.sql.functions import lag, col, Column, when, isnull, to_date, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql.window import Window

In [4]:
import yfinance as yf
tsla = yf.Ticker(ticker)
tsla.info

{'zip': '94304',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 48016,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, Netherlands, Norway, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers sedans and sport utility vehicles. It also provides electric powertrain components and systems; and services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians, as well as sells used vehicles. This segment markets and sells its products through a network of company-owned stores and galleries, as well as through its own Website. The Energy Generation and Storage segment offers energy storage products, such as rechargeable lithium-ion battery systems for use in homes, industrial, commercial facilities, and utility grids; and desig

In [5]:
df = tsla.history(period="30d")
df.describe

<bound method NDFrame.describe of               Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                         
2020-04-13  590.16  652.00  580.53  650.95  22475400          0             0
2020-04-14  698.97  741.88  692.43  709.89  30576500          0             0
2020-04-15  742.00  753.13  710.00  729.83  23577000          0             0
2020-04-16  716.94  759.45  706.72  745.21  20657900          0             0
2020-04-17  772.28  774.95  747.66  753.89  13128200          0             0
2020-04-20  732.70  765.57  712.21  746.36  14746600          0             0
2020-04-21  730.12  753.33  673.79  686.72  20209100          0             0
2020-04-22  703.98  734.00  688.71  732.11  14224800          0             0
2020-04-23  727.60  734.00  703.13  705.63  13236700          0             0
2020-04-24  710.81  730.73  698.18  725.15  13237600          0             0
2020-04-27  737.61  799.49  73

In [6]:
#sparkdf = spark.createDataFrame(df)
#sparkdf.show()

In [7]:
#spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkdf = sqlContext.createDataFrame(df.reset_index (drop=False))
sparkdf.show()

+-------------------+------+------+------+------+--------+---------+------------+
|               Date|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+-------------------+------+------+------+------+--------+---------+------------+
|2020-04-13 00:00:00|590.16| 652.0|580.53|650.95|22475400|        0|           0|
|2020-04-14 00:00:00|698.97|741.88|692.43|709.89|30576500|        0|           0|
|2020-04-15 00:00:00| 742.0|753.13| 710.0|729.83|23577000|        0|           0|
|2020-04-16 00:00:00|716.94|759.45|706.72|745.21|20657900|        0|           0|
|2020-04-17 00:00:00|772.28|774.95|747.66|753.89|13128200|        0|           0|
|2020-04-20 00:00:00| 732.7|765.57|712.21|746.36|14746600|        0|           0|
|2020-04-21 00:00:00|730.12|753.33|673.79|686.72|20209100|        0|           0|
|2020-04-22 00:00:00|703.98| 734.0|688.71|732.11|14224800|        0|           0|
|2020-04-23 00:00:00| 727.6| 734.0|703.13|705.63|13236700|        0|           0|
|2020-04-24 00:0

In [8]:
#df_pandas = df_pandas.drop(['Dividends', 'Stock Splits'], axis=1)
sparkdf = sparkdf.drop('Dividends','Stock Splits')
sparkdf.show()

+-------------------+------+------+------+------+--------+
|               Date|  Open|  High|   Low| Close|  Volume|
+-------------------+------+------+------+------+--------+
|2020-04-13 00:00:00|590.16| 652.0|580.53|650.95|22475400|
|2020-04-14 00:00:00|698.97|741.88|692.43|709.89|30576500|
|2020-04-15 00:00:00| 742.0|753.13| 710.0|729.83|23577000|
|2020-04-16 00:00:00|716.94|759.45|706.72|745.21|20657900|
|2020-04-17 00:00:00|772.28|774.95|747.66|753.89|13128200|
|2020-04-20 00:00:00| 732.7|765.57|712.21|746.36|14746600|
|2020-04-21 00:00:00|730.12|753.33|673.79|686.72|20209100|
|2020-04-22 00:00:00|703.98| 734.0|688.71|732.11|14224800|
|2020-04-23 00:00:00| 727.6| 734.0|703.13|705.63|13236700|
|2020-04-24 00:00:00|710.81|730.73|698.18|725.15|13237600|
|2020-04-27 00:00:00|737.61|799.49| 735.0|798.75|20681400|
|2020-04-28 00:00:00|795.64| 805.0|756.69|769.12|15222000|
|2020-04-29 00:00:00|790.17| 803.2|783.16|800.51|16216000|
|2020-04-30 00:00:00|855.19|869.82| 763.5|781.88|2847190

In [9]:
#sparkdf = sparkdf.withColumn('Date',to_date(col('date'), "yyyy-MM-dd")).show()
sparkdf = sparkdf.withColumn("Date", sparkdf["Date"].cast(DateType()))
#sparkdf

In [10]:
#sparkdf = sqlContext.createDataFrame(sparkdf.reset_index (drop=False))

In [11]:
sparkdf.show()

+----------+------+------+------+------+--------+
|      Date|  Open|  High|   Low| Close|  Volume|
+----------+------+------+------+------+--------+
|2020-04-13|590.16| 652.0|580.53|650.95|22475400|
|2020-04-14|698.97|741.88|692.43|709.89|30576500|
|2020-04-15| 742.0|753.13| 710.0|729.83|23577000|
|2020-04-16|716.94|759.45|706.72|745.21|20657900|
|2020-04-17|772.28|774.95|747.66|753.89|13128200|
|2020-04-20| 732.7|765.57|712.21|746.36|14746600|
|2020-04-21|730.12|753.33|673.79|686.72|20209100|
|2020-04-22|703.98| 734.0|688.71|732.11|14224800|
|2020-04-23| 727.6| 734.0|703.13|705.63|13236700|
|2020-04-24|710.81|730.73|698.18|725.15|13237600|
|2020-04-27|737.61|799.49| 735.0|798.75|20681400|
|2020-04-28|795.64| 805.0|756.69|769.12|15222000|
|2020-04-29|790.17| 803.2|783.16|800.51|16216000|
|2020-04-30|855.19|869.82| 763.5|781.88|28471900|
|2020-05-01| 755.0|772.77|683.04|701.32|32531800|
|2020-05-04| 701.0| 762.0| 698.0|761.19|19237100|
|2020-05-05|789.79|798.92|762.18|768.21|16991700|


In [12]:
from pyspark.sql.functions import lag, col, when, isnull

from pyspark.sql.window import Window

In [13]:
w=Window().partitionBy().orderBy(col("Date"))
#sparkdf.select("*", lag("Close").over(w).alias("newcolumn")).na.drop().show()
sparkdf = sparkdf.withColumn("PreviousClose", lag("Close",1,0).over(w))

In [14]:
sparkdf.show()

+----------+------+------+------+------+--------+-------------+
|      Date|  Open|  High|   Low| Close|  Volume|PreviousClose|
+----------+------+------+------+------+--------+-------------+
|2020-04-13|590.16| 652.0|580.53|650.95|22475400|          0.0|
|2020-04-14|698.97|741.88|692.43|709.89|30576500|       650.95|
|2020-04-15| 742.0|753.13| 710.0|729.83|23577000|       709.89|
|2020-04-16|716.94|759.45|706.72|745.21|20657900|       729.83|
|2020-04-17|772.28|774.95|747.66|753.89|13128200|       745.21|
|2020-04-20| 732.7|765.57|712.21|746.36|14746600|       753.89|
|2020-04-21|730.12|753.33|673.79|686.72|20209100|       746.36|
|2020-04-22|703.98| 734.0|688.71|732.11|14224800|       686.72|
|2020-04-23| 727.6| 734.0|703.13|705.63|13236700|       732.11|
|2020-04-24|710.81|730.73|698.18|725.15|13237600|       705.63|
|2020-04-27|737.61|799.49| 735.0|798.75|20681400|       725.15|
|2020-04-28|795.64| 805.0|756.69|769.12|15222000|       798.75|
|2020-04-29|790.17| 803.2|783.16|800.51|

In [15]:

sparkdf = sparkdf.withColumn("Difference", when(isnull(sparkdf.Close - sparkdf.PreviousClose),0).otherwise(sparkdf.Close-sparkdf.PreviousClose))


In [16]:
sparkdf.show()

+----------+------+------+------+------+--------+-------------+-------------------+
|      Date|  Open|  High|   Low| Close|  Volume|PreviousClose|         Difference|
+----------+------+------+------+------+--------+-------------+-------------------+
|2020-04-13|590.16| 652.0|580.53|650.95|22475400|          0.0|             650.95|
|2020-04-14|698.97|741.88|692.43|709.89|30576500|       650.95|  58.93999999999994|
|2020-04-15| 742.0|753.13| 710.0|729.83|23577000|       709.89| 19.940000000000055|
|2020-04-16|716.94|759.45|706.72|745.21|20657900|       729.83| 15.379999999999995|
|2020-04-17|772.28|774.95|747.66|753.89|13128200|       745.21|   8.67999999999995|
|2020-04-20| 732.7|765.57|712.21|746.36|14746600|       753.89| -7.529999999999973|
|2020-04-21|730.12|753.33|673.79|686.72|20209100|       746.36|-59.639999999999986|
|2020-04-22|703.98| 734.0|688.71|732.11|14224800|       686.72| 45.389999999999986|
|2020-04-23| 727.6| 734.0|703.13|705.63|13236700|       732.11|-26.480000000

In [17]:
sparkdf = sparkdf.withColumn("percentage_change", when(isnull(sparkdf.Difference / sparkdf.PreviousClose),0).otherwise(sparkdf.Difference / sparkdf.PreviousClose))


In [18]:
sparkdf.show(31)

+----------+------+------+------+------+--------+-------------+-------------------+--------------------+
|      Date|  Open|  High|   Low| Close|  Volume|PreviousClose|         Difference|   percentage_change|
+----------+------+------+------+------+--------+-------------+-------------------+--------------------+
|2020-04-13|590.16| 652.0|580.53|650.95|22475400|          0.0|             650.95|                 0.0|
|2020-04-14|698.97|741.88|692.43|709.89|30576500|       650.95|  58.93999999999994| 0.09054458867808578|
|2020-04-15| 742.0|753.13| 710.0|729.83|23577000|       709.89| 19.940000000000055|0.028088858837284725|
|2020-04-16|716.94|759.45|706.72|745.21|20657900|       729.83| 15.379999999999995|0.021073400654947037|
|2020-04-17|772.28|774.95|747.66|753.89|13128200|       745.21|   8.67999999999995|0.011647723460500999|
|2020-04-20| 732.7|765.57|712.21|746.36|14746600|       753.89| -7.529999999999973|-0.00998819456419...|
|2020-04-21|730.12|753.33|673.79|686.72|20209100|      

In [19]:
#sparkdf.write.csv('yfinance.csv')

In [20]:
sparkdf.write.format('jdbc').options(
        url='jdbc:mysql://localhost:3306/stocks?serverTimezone=UTC',
        driver='com.mysql.jdbc.Driver',
        dbtable='yfinance',
        user='root',
        password='yourpassword').mode('ignore').save()